In [1]:
import pandas as pd

# Load the datasets
price_df = pd.read_csv('Phase 0 - Price.csv')
sales_df = pd.read_csv('Phase 0 - Sales.csv')

# Inspect the first few rows
print(price_df.head())
print(sales_df.head())


   Client  Warehouse  Product  2020-07-06  2020-07-13  2020-07-20  2020-07-27  \
0       0          1      367   10.900001   10.900001   10.900001   15.582857   
1       0          1      639         NaN         NaN         NaN         NaN   
2       0          1      655   21.343332   21.343332   21.343332   22.881200   
3       0          1     1149   11.480000   11.480000   11.480000   12.291429   
4       0          1     1485         NaN         NaN         NaN         NaN   

   2020-08-03  2020-08-10  2020-08-17  ...  2023-07-31  2023-08-07  \
0   27.289999   27.289999   27.289999  ...   52.940000         NaN   
1         NaN         NaN         NaN  ...  106.900000   99.866000   
2   25.188000   25.188000   25.188000  ...   30.794443   29.555000   
3   14.320000   14.320000   14.320000  ...   34.980000   34.980000   
4         NaN         NaN         NaN  ...   29.936190   29.583042   

   2023-08-14  2023-08-21  2023-08-28  2023-09-04  2023-09-11  2023-09-18  \
0   52.940000  

In [2]:
# Melt the data to reshape it into long format for easier time series handling
price_df_melt = pd.melt(price_df, id_vars=['Client', 'Warehouse', 'Product'], var_name='date', value_name='price')
sales_df_melt = pd.melt(sales_df, id_vars=['Client', 'Warehouse', 'Product'], var_name='date', value_name='sales')

# Merge the datasets on Client, Warehouse, Product, and date
merged_df = pd.merge(sales_df_melt, price_df_melt, on=['Client', 'Warehouse', 'Product', 'date'], how='left')

# Convert 'date' column to datetime type
merged_df['date'] = pd.to_datetime(merged_df['date'])

# Fill missing values in sales with 0
merged_df['sales'].fillna(0, inplace=True)

# Inspect merged dataset
print(merged_df.head())


   Client  Warehouse  Product       date  sales      price
0       0          1      367 2020-07-06    7.0  10.900001
1       0          1      639 2020-07-06    0.0        NaN
2       0          1      655 2020-07-06   21.0  21.343332
3       0          1     1149 2020-07-06    7.0  11.480000
4       0          1     1485 2020-07-06    0.0        NaN


In [3]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np

# Get the unique combinations of Client, Warehouse, and Product
unique_products = merged_df[['Client', 'Warehouse', 'Product']].drop_duplicates()

# Create a DataFrame to store the results
future_sales_df = pd.DataFrame()

# Set future dates for the next 13 weeks
future_dates = pd.date_range(start='2023-10-09', periods=13, freq='W-MON')

for _, row in unique_products.iterrows():
    client, warehouse, product = row['Client'], row['Warehouse'], row['Product']
    
    # Filter the data for the current product
    product_data = merged_df[(merged_df['Client'] == client) & 
                             (merged_df['Warehouse'] == warehouse) & 
                             (merged_df['Product'] == product)].sort_values('date')
    
    # Get the sales data
    sales_data = product_data['sales'].values
    
    # Train the Exponential Smoothing model (You can also experiment with other models)
    model = ExponentialSmoothing(sales_data, trend='add', seasonal='add', seasonal_periods=4).fit()
    
    # Forecast sales for the next 13 weeks
    future_sales = model.forecast(13)
    
    # Create a DataFrame to store the results for this product
    product_forecast = pd.DataFrame({
        'Client': client,
        'Warehouse': warehouse,
        'Product': product,
        'date': future_dates,
        'forecasted_sales': future_sales
    })
    
    # Append the forecast to the future_sales_df
    future_sales_df = pd.concat([future_sales_df, product_forecast], ignore_index=True)

# Pivot the DataFrame to get the required format
forecast_pivot_df = future_sales_df.pivot_table(index=['Client', 'Warehouse', 'Product'], 
                                                columns='date', 
                                                values='forecasted_sales').reset_index()

# Rename the columns to match the required format
forecast_pivot_df.columns = ['Client', 'Warehouse', 'Product'] + [date.strftime('%Y-%m-%d') for date in future_dates]

# Display the final DataFrame
print(forecast_pivot_df.head())

# Save the results to a CSV file
forecast_pivot_df.to_csv('sales_forecast.csv', index=False)
print("Forecast saved to 'sales_forecast.csv'")


/usr/lib/python3/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/lib/python3/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/lib/python3/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/lib/python3/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/lib/python3/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/lib/python3/dist-packages/statsmodels/tsa/holtwinters/model.py:917: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
/usr/lib/python3/dist-packag

   Client  Warehouse  Product  2023-10-09  2023-10-16  2023-10-23  2023-10-30  \
0       0          1      367    1.785267    1.642415    1.685440    1.220327   
1       0          1      639    5.878874    5.855120    6.047489    6.256668   
2       0          1      655   27.354430   27.663960   26.762962   26.739751   
3       0          1     1149    1.165645    1.094239    0.713176    0.527140   
4       0          1     1485    7.895543    9.052072    9.906627    9.759413   

   2023-11-06  2023-11-13  2023-11-20  2023-11-27  2023-12-04  2023-12-11  \
0    1.657197    1.514344    1.557370    1.092257    1.529126    1.386274   
1    6.021367    5.997614    6.189982    6.399162    6.163860    6.140107   
2   27.468899   27.778429   26.877431   26.854220   27.583368   27.892898   
3    1.019312    0.947906    0.566842    0.380806    0.872978    0.801572   
4    8.724459    9.880988   10.735543   10.588329    9.553375   10.709904   

   2023-12-18  2023-12-25  2024-01-01  
0    1.429